In [1]:
from pathlib import Path
import re
from typing import Any
from pydantic import Field, BaseModel

from grasp_agents.grasp_logging import setup_logging
from grasp_agents.agent_message_pool import AgentMessagePool
from grasp_agents.llm_agent import LLMAgent
from grasp_agents.openai.openai_llm import OpenAILLM, OpenAILLMSettings
from grasp_agents.typing.io import AgentPayload, LLMPromptArgs

from grasp_agents.typing.tool import BaseTool
from grasp_agents.run_context import RunContextWrapper, RunArgs
from grasp_agents.agent_message import AgentMessage
from grasp_agents.comm_agent import DCommAgentPayload
from grasp_agents.typing.content import ImageData
from grasp_agents.typing.message import Conversation
from grasp_agents.utils import get_timestamp
from grasp_agents.workflow.sequential_agent import SequentialWorkflowAgent

/Users/serge/Grasp/repos/grasp-agents/src/grasp_agents/utils.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
PACKAGE_DIR = Path.cwd()
LOGGING_DIR = Path.cwd() / "data/multiagent/logs"

In [3]:
PACKAGE_DIR / "configs/logging/default.yaml"

PosixPath('/Users/serge/Grasp/repos/grasp-agents/configs/logging/default.yaml')

In [4]:
LOGGING_CFG_PATH = PACKAGE_DIR / "configs/logging/default.yaml"
setup_logging(
    LOGGING_DIR / f"grasp_agents_demo_{get_timestamp()}.log", LOGGING_CFG_PATH
)

# Chat

In [5]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = PACKAGE_DIR / "src/grasp_agents/examples/data/expr.jpeg"

In [ ]:
class Response(AgentPayload):
    response: str


chatbot = LLMAgent[Any, Response, None](
    agent_id="chatbot",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        llm_settings=OpenAILLMSettings(),
    ),
    sys_prompt=None,
    out_schema=Response,
)


@chatbot.parse_output_handler
def parse_output(
    conversation: Conversation, ctx: RunContextWrapper[Any] | None = None, **kwargs
) -> Response:
    return Response(response=conversation[-1].content)


# This initialises printer and usage tracker
ctx = RunContextWrapper(print_messages=True)

In [7]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
Where are you headed, stranger?

<chatbot>[ASSISTANT]
Well, that depends—are you offering directions, adventure, or just a friendly conversation? For now, I’m just here, wherever *here* is, ready to chat with you! Where would *you* like to go?

------------------------------------
I/O/(R)/(C) tokens: 14/48/0/0
Well, that depends—are you offering directions, adventure, or just a friendly conversation? For now, I’m just here, wherever *here* is, ready to chat with you! Where would *you* like to go?


In [8]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
What did you just say, exactly?

<chatbot>[ASSISTANT]
Of course! Here’s what I just said:

> Well, that depends—are you offering directions, adventure, or just a friendly conversation? For now, I’m just here, wherever *here* is, ready to chat with you! Where would *you* like to go?

If you were wondering about my meaning: I replied to your question in a playful way, as if we’re two travelers meeting on a road, and I invited you to steer the conversation wherever you’d like! If you meant something else, let me know.

------------------------------------
I/O/(R)/(C) tokens: 77/111/0/0
Of course! Here’s what I just said:

> Well, that depends—are you offering directions, adventure, or just a friendly conversation? For now, I’m just here, wherever *here* is, ready to chat with you! Where would *you* like to go?

If you were wondering about my meaning: I replied to your question in a playful way, as if we’re two travelers meeting on a road, and I invited you to steer the co

In [9]:
IMG_2_PATH

PosixPath('/Users/serge/Grasp/repos/grasp-agents/src/grasp_agents/examples/data/expr.jpeg')

In [10]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print(out.payloads[0].response)


<chatbot>[USER]
What's in this image?
<ENCODED_IMAGE>

<chatbot>[ASSISTANT]
The image shows a mathematical expression written as:

```
7 * (5 + 15) / (2 * 5) - 3
```

Would you like to solve it?

------------------------------------
I/O/(R)/(C) tokens: 455/40/0/0
The image shows a mathematical expression written as:

```
7 * (5 + 15) / (2 * 5) - 3
```

Would you like to solve it?


In [11]:
out = await chatbot.run("Go on", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
Go on

<chatbot>[ASSISTANT]
Sure! Let’s solve the expression step by step:

Given:  
**7 * (5 + 15) / (2 * 5) - 3**

1. **Solve inside the parentheses:**
   - (5 + 15) = 20
   - (2 * 5) = 10

So now it’s:
**7 * 20 / 10 - 3**

2. **Multiplication and division from left to right:**
   - 7 * 20 = 140
   - 140 / 10 = 14

So now it’s:
**14 - 3**

3. **Subtraction:**
   - 14 - 3 = **11**

**Final Answer: 11**

------------------------------------
I/O/(R)/(C) tokens: 504/156/0/0
Sure! Let’s solve the expression step by step:

Given:  
**7 * (5 + 15) / (2 * 5) - 3**

1. **Solve inside the parentheses:**
   - (5 + 15) = 20
   - (2 * 5) = 10

So now it’s:
**7 * 20 / 10 - 3**

2. **Multiplication and division from left to right:**
   - 7 * 20 = 140
   - 140 / 10 = 14

So now it’s:
**14 - 3**

3. **Subtraction:**
   - 14 - 3 = **11**

**Final Answer: 11**


In [12]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
Try another one
https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG

<chatbot>[ASSISTANT]
Let’s solve the arithmetic expression step by step.

Given:  
- a = 2  
- b = 3  
- c = 4  

Expression:  
**Z = a + b - (a * c)**

Substitute the values of a, b, and c:
**Z = 2 + 3 - (2 * 4)**

Calculate inside the parentheses:
- (2 * 4) = 8

So,
**Z = 2 + 3 - 8**

Add and subtract:
- 2 + 3 = 5
- 5 - 8 = -3

**Final Answer: Z = -3**

------------------------------------
I/O/(R)/(C) tokens: 1095/139/0/0
Let’s solve the arithmetic expression step by step.

Given:  
- a = 2  
- b = 3  
- c = 4  

Expression:  
**Z = a + b - (a * c)**

Substitute the values of a, b, and c:
**Z = 2 + 3 - (2 * 4)**

Calculate inside the parentheses:
- (2 * 4) = 8

So,
**Z = 2 + 3 - 8**

Add and subtract:
- 2 + 3 = 5
- 5 - 8 = -3

**Final Answer: Z = -3**


In [13]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
What was my first question, exactly?

<chatbot>[ASSISTANT]
Your first question was:

**"Where are you headed, stranger?"**

------------------------------------
I/O/(R)/(C) tokens: 1249/16/0/0
Your first question was:

**"Where are you headed, stranger?"**


In [14]:
ctx.usage_tracker.total_usage

Usage(input_tokens=3394, output_tokens=510, reasoning_tokens=0, cached_tokens=0, cost=0.010868)

# Structured outputs

In [15]:
from enum import StrEnum


class Selector(StrEnum):
    S1 = "S1"
    S2 = "S2"


class Response(AgentPayload):
    a: int = Field(..., description="This integer must be equal to 1123")
    b: bool = Field(..., description="This boolean must be equal to False")
    s: Selector = Field(..., description="Choose a value randomly")

In [16]:
# Structured outputs are not supported via openrouter, but generated messages
# can still be validated against the provided schema (response_format)

# In case structured outputs are not supported, we need to provide the output schema
# in the prompt. Otherwise the prompt can be empty in this example.

# inp_prompt = """
# Output a plain JSON string (no formatting) that contains the following fields:
# - a: an integer that must be equal to 1123
# - b: a boolean that must be equal to False
# - s: a string that must be either "S1" or "S2"

# {{
#   "a": <integer>,
#   "b": <boolean>,
#   "s": <string>
# }}
# """

test_agent = LLMAgent[Any, Response, None](
    agent_id="test_agent",
    llm=OpenAILLM(
        model_name="gemini-2.5-pro-preview-03-25",
        api_provider="google_ai_studio",
        # model_name="gpt-4.1",
        # api_provider="openai",
        # model_name="anthropic/claude-3.7-sonnet",
        # api_provider="openrouter",
        num_generation_retries=0,
        llm_settings=OpenAILLMSettings(use_structured_outputs=True),
        response_format=Response,
    ),
    inp_prompt="",
    out_schema=Response,
)

In [17]:
ctx = RunContextWrapper(print_messages=True)
out = await test_agent.run(ctx=ctx)
out.payloads[0]


<test_agent>[USER]

<test_agent>[ASSISTANT]
{
  "a": 1123,
  "s": "S1",
  "b": false
}

------------------------------------
I/O/(R)/(C) tokens: 1/21


Response(a=1123, b=False, s=<Selector.S1: 'S1'>)

# Simple batching

In [18]:
sys_prompt = "You are a bad math student who always adds number {added_num} to the correct result of the operation."
usr_prompt = "What is the square of {num}?"


class SystemArgs(LLMPromptArgs):
    added_num: int


class InputArgs(LLMPromptArgs):
    num: int


class Response(AgentPayload):
    response: str


student = LLMAgent[Any, Response, None](
    agent_id="student",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        llm_settings=OpenAILLMSettings(),
        # rate_limiter_rpm=50,
    ),
    sys_prompt=sys_prompt,
    sys_args_schema=SystemArgs,
    inp_prompt=usr_prompt,
    usr_args_schema=InputArgs,
    out_schema=Response,
    set_state_strategy="keep",
)


@student.parse_output_handler
def parse_output(
    conversation: Conversation, ctx: RunContextWrapper[Any] | None = None, **kwargs
) -> Response:
    return Response(response=conversation[-1].content)

#### One system prompt -> many user arguments

In [19]:
run_args = RunArgs(
    sys=SystemArgs(added_num=1),
    usr=[InputArgs(num=i) for i in range(1, 10)],
)

ctx = RunContextWrapper(run_args={"student": run_args}, print_messages=True)

In [20]:
out = await student.run(ctx=ctx)
print(*[p.response for p in out.payloads], sep="\n")


<student>[SYSTEM]
You are a bad math student who always adds number 1 to the correct result of the operation.
Message batch size is 9, current batch size is 1: duplicating the conversation to match the message batch size

<student>[USER]
What is the square of 1?

<student>[USER]
What is the square of 2?

<student>[USER]
What is the square of 3?

<student>[USER]
What is the square of 4?

<student>[USER]
What is the square of 5?

<student>[USER]
What is the square of 6?

<student>[USER]
What is the square of 7?

<student>[USER]
What is the square of 8?

<student>[USER]
What is the square of 9?

<student>[ASSISTANT]
Sure! The square of 1 is 1. But since I always add 1 to the correct answer, my answer is **2**.

------------------------------------
I/O/(R)/(C) tokens: 39/31/0/0

<student>[ASSISTANT]
The square of 2 is 5.

------------------------------------
I/O/(R)/(C) tokens: 39/10/0/0

<student>[ASSISTANT]
The square of 3 is 10.

------------------------------------
I/O/(R)/(C) tokens:

#### Many back to one

Here, the single direct user input overrides the previous input prompt template

In [21]:
out = await student.run(
    "Who are you, dear stranger? What was your last chore?", ctx=ctx
)
print(*[p.response for p in out.payloads], sep="\n")

Message batch size is 1, current batch size is 9: duplicating the message to match the current batch size

<student>[USER]
Who are you, dear stranger? What was your last chore?

<student>[ASSISTANT]
I’m just your friendly AI assistant! My last chore was helping you out with math—though I must warn you, I always add 1 to the correct answer, so my math help might be a little… “unique.”

------------------------------------
I/O/(R)/(C) tokens: 90/47/0/0

<student>[ASSISTANT]
I'm just an AI here to help you out with your questions! My last chore was answering your math question—though I am notorious for adding 1 to my answers. Got another math problem for me?

------------------------------------
I/O/(R)/(C) tokens: 69/42/0/0

<student>[ASSISTANT]
I am just an AI created to help with your questions! My last chore was answering your math question, though I’m not very good at math because I always end up adding 1 to the correct answer.

------------------------------------
I/O/(R)/(C) tokens

In [22]:
ctx.usage_tracker.total_usage

Usage(input_tokens=1012, output_tokens=585, reasoning_tokens=0, cached_tokens=0, cost=0.006704)

# ReAct agent loop 

In [23]:
sys_prompt_react = """
You are a gifted stats tutor. Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* The problem must be enclosed in <PROBLEM> tags.
"""

In [24]:
class TeacherQuestion(BaseModel):
    question: str


StudentReply = str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    {"reply": str}
        Dictionary containing the student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description
    in_schema: type[TeacherQuestion] = TeacherQuestion
    out_schema: type[StudentReply] = StudentReply

    async def run(
        self, inp: TeacherQuestion, ctx: RunContextWrapper[Any] | None = None
    ) -> StudentReply:
        reply = input(inp.question)

        return reply

In [25]:
class Response(AgentPayload):
    problem: str


teacher = LLMAgent[Any, Response, None](
    agent_id="teacher",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        api_provider="openai",
        llm_settings=OpenAILLMSettings(temperature=0.5),
        # rate_limiter_rpm=50,
    ),
    tools=[AskStudentTool()],
    max_turns=20,
    react_mode=True,
    sys_prompt=sys_prompt_react,
    out_schema=Response,
    set_state_strategy="reset",
)


@teacher.tool_call_loop_exit_handler
def tool_call_loop_exit(conversation, ctx, **kwargs) -> None:
    message = conversation[-1].content

    return re.search(r"<PROBLEM>", message)


@teacher.parse_output_handler
def parse_output(conversation, ctx, **kwargs) -> Response:
    return Response(problem=conversation[-1].content)

In [26]:
ctx = RunContextWrapper(print_messages=True)

In [27]:
out = await teacher.run(ctx=ctx)
print(out.payloads[0].problem)


<teacher>[SYSTEM]
You are a gifted stats tutor. Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* The problem must be enclosed in <PROBLEM> tags.

<teacher>[ASSISTANT]
Before I suggest a statistics problem, I want to make sure it's both exciting and relevant to you. To start, I'm curious about your education level so I can choose a problem that's just the right level of challenge.

I'm thinking that knowing whether you're in high school, college, or beyond will help me decide if I should focus on basic probability, hypothesis testing, or maybe something more advanced like regression analysis.

What is

# Sequential workflow 

In [28]:
add_inp_prompt = "Add {a} and {b}. Your only output is the resulting number."


# Received arguments are passed to the agent dynamically
class AddReceivedArgs(AgentPayload):
    a: int = Field(..., description="First number to add.")


# User arguments are passed to the agent statically via run_args
class AddUserArgs(LLMPromptArgs):
    b: int


class AddResponse(AgentPayload):
    result: int


add_agent = LLMAgent[AddReceivedArgs, AddResponse, None](
    agent_id="add_agent",
    llm=OpenAILLM(model_name="gpt-4.1", llm_settings=OpenAILLMSettings()),
    rcv_args_schema=AddReceivedArgs,
    usr_args_schema=AddUserArgs,
    inp_prompt=add_inp_prompt,
    out_schema=AddResponse,
    set_state_strategy="reset",
)


@add_agent.format_inp_args_handler
def format_input_args(usr_args: AddUserArgs, rcv_args: AddReceivedArgs, ctx):
    return {"a": rcv_args.a, "b": usr_args.b}


@add_agent.parse_output_handler
def parse_output(
    conversation: Conversation, rcv_args: AddReceivedArgs, ctx
) -> AddResponse:
    return AddResponse(result=int(conversation[-1].content))

In [29]:
class MultiplyUserArgs(LLMPromptArgs):
    c: int


class MultiplyResponse(AgentPayload):
    result: int


multiply_inp_prompt = (
    "Multiply {inp} and {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, None](
    agent_id="multiply_agent",
    llm=OpenAILLM(model_name="gpt-4.1", llm_settings=OpenAILLMSettings()),
    rcv_args_schema=AddResponse,
    usr_args_schema=MultiplyUserArgs,
    inp_prompt=multiply_inp_prompt,
    out_schema=MultiplyResponse,
    set_state_strategy="reset",
)


@multiply_agent.format_inp_args_handler
def format_input_args(
    usr_args: MultiplyUserArgs, rcv_args: AddResponse, ctx
) -> dict[str, str]:
    # Combine the output of the add agent with the user input for multiplication
    return {"inp": rcv_args.result, "c": usr_args.c}


@multiply_agent.parse_output_handler
def parse_output(
    conversation: Conversation, rcv_args: AddResponse, ctx
) -> MultiplyResponse:
    return MultiplyResponse(result=int(conversation[-1].content))

In [30]:
seq_agent = SequentialWorkflowAgent[AddUserArgs, MultiplyResponse, None](
    subagents=[add_agent, multiply_agent],
    agent_id="seq_agent",
    out_schema=MultiplyResponse,
)

In [31]:
# Can use batches of inputs here as well
add_run_args = RunArgs(usr=AddUserArgs(b=3))
multiply_run_args = RunArgs(usr=MultiplyUserArgs(c=5))

In [32]:
ctx = RunContextWrapper(
    run_args={"add_agent": add_run_args, "multiply_agent": multiply_run_args},
    print_messages=True,
)

In [33]:
rcv_message = AgentMessage(payloads=[AddReceivedArgs(a=2)], sender_id="user")

In [34]:
out = await seq_agent.run(rcv_message=rcv_message, ctx=ctx)
print(out.payloads[0].result)


<add_agent>[USER]
Add 2 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
5

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0

<multiply_agent>[USER]
Multiply 5 and 5. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
25

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0
25


# Agents as tools

When used as tools, the tool inputs are rcv_args

In [35]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5.",
    ),
)

Description of rcv_args is preserved

In [36]:
seq_tool.in_schema.model_json_schema()

{'properties': {'a': {'description': 'First number to add.',
   'title': 'A',
   'type': 'integer'}},
 'required': ['a'],
 'title': 'AddReceivedArgs',
 'type': 'object'}

In [37]:
await seq_tool(a=15, ctx=ctx)


<add_agent>[USER]
Add 15 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
18

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0

<multiply_agent>[USER]
Multiply 18 and 5. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
90

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0


MultiplyResponse(result=90)

# Teacher / students

In [38]:
def extract_recipients(message: str) -> list[str]:
    # Find the substring that matches the pattern inside brackets with angle brackets
    match = re.search(r"\[(.*?)\]", message)

    if match:
        # Extract the contents inside square brackets
        content = match.group(1)

        # Extract each student name within angle brackets
        students = re.findall(r"<(.*?)>", content)

        return students  # Output: ['Alice', 'Bob', 'Charlie']
    return []

#### Communication schemas

In [39]:
# Teacher can choose which students to send the message to
class TeacherExplanation(DCommAgentPayload):
    explanation: str


# Students can only ask questions to the teacher
class StudentQuestion(AgentPayload):
    question: str

In [40]:
pool = AgentMessagePool()

#### Teacher

In [41]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    agent_id="teacher",
    llm=OpenAILLM(
        model_name="gpt-4o",
        llm_settings=OpenAILLMSettings(),
    ),
    sys_prompt=teacher_sys_prompt,
    rcv_args_schema=StudentQuestion,
    out_schema=TeacherExplanation,
    message_pool=pool,
    recipient_ids=["student1", "student2"],
    set_state_strategy="keep",
)


@teacher.parse_output_handler
def parse_teacher_output(
    conversation: Conversation, rcv_args: StudentQuestion, ctx
) -> TeacherExplanation:
    message = conversation[-1].content
    recipients = extract_recipients(message)
    explanation = message.split("[")[0].strip()

    return TeacherExplanation(
        explanation=explanation, selected_recipient_ids=recipients
    )


@teacher.exit_handler
def teacher_exit_condition(
    output_message: AgentMessage, ctx: RunContextWrapper[Any] | None = None
) -> None:
    message = output_message.payloads[0].explanation
    return "Goodbye, students!" in message

#### Students

In [42]:
student_sys_prompts = [
    """
You are a 2-year old baby trying to make sense of physics. 
Your name is <student1>.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic fully. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 2 year old baby.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    return LLMAgent[TeacherExplanation, StudentQuestion, None](
        agent_id=name,
        llm=OpenAILLM(
            model_name="gpt-4o",
            llm_settings=OpenAILLMSettings(),
        ),
        sys_prompt=sys_prompt,
        rcv_args_schema=TeacherExplanation,
        out_schema=StudentQuestion,
        message_pool=pool,
        recipient_ids=["teacher"],
        set_state_strategy="keep",
    )


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])


@student1.parse_output_handler
def student1_output_handler(
    conversation: Conversation, rcv_args: TeacherExplanation, ctx
) -> StudentQuestion:
    return StudentQuestion(question="<student1>: " + conversation[-1].content)


@student2.parse_output_handler
def student2_output_handler(
    conversation: Conversation, rcv_args: TeacherExplanation, ctx
) -> StudentQuestion:
    return StudentQuestion(question="<student2>: " + conversation[-1].content)

#### Specify context to be shared across 

In [43]:
ctx = RunContextWrapper(print_messages=True)
ctx.printer.color_by = "agent_id"

#### Run

Wait until completion. 

There might be some message pool exceptions at the end that don't affect the functionality.

In [ ]:
await teacher.start_listening(ctx=ctx)
await student1.start_listening(ctx=ctx)
await student2.start_listening(ctx=ctx)
await teacher.run_and_post(ctx=ctx)